In [131]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [132]:
import seaborn as sns

In [133]:
df = sns.load_dataset('titanic')

In [134]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [135]:
df.shape

(891, 15)

In [136]:
df.drop(columns=['who','adult_male','class','deck','embark_town','alive','alone'],inplace=True)

In [118]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [192]:
#Train-test-Split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['survived']),df['survived'],random_state = 42, test_size=0.2)

In [193]:
X_train

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [194]:
y_train

,survived
331,0
733,0
382,0
704,0
813,0
...,...
106,1
270,0
860,0
435,1


In [195]:
df.isnull().sum()

,0
survived,0
pclass,0
sex,0
age,177
sibsp,0
parch,0
fare,0
embarked,2


In [196]:
#Simple Imputer transformer for the missing values-replacing with mean.
trf1= ColumnTransformer(transformers=[
    ('impute age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])],remainder='passthrough')

In [229]:
pipe1 = make_pipeline(trf1)
X_train_imputed = pipe1.fit_transform(X_train)

In [231]:
X_train_imputed[0]

array([45.5, 'S', 1, 'male', 0, 0, 28.5], dtype=object)

In [232]:
X_train

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [233]:
#One Hot Encoding
trf2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked',OneHotEncoder( drop='first',sparse_output=False,handle_unknown='ignore'),[3,1])],remainder='passthrough')

In [245]:
# Scaling- we are not using standarsition and using Min max Scaler here because we want to do feature selection and for that we will be using specifically DT
trf3 = ColumnTransformer(transformers=[
    ('scale',MinMaxScaler(),[0,-1])],remainder='passthrough')

In [246]:
#Feature selection-we just need to call function SelectKbest
trf4 = SelectKBest(score_func=chi2,k=8)

In [247]:
# Decision Tree
trf5 = DecisionTreeClassifier()

##**Pipeline**

In [248]:
#We will now join all the indvidual transformers into a pipeline
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [249]:
pipe.steps

[('columntransformer-1',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('impute age', SimpleImputer(), [2]),
                                  ('impute_embarked',
                                   SimpleImputer(strategy='most_frequent'),
                                   [6])])),
 ('columntransformer-2',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('ohe_sex_embarked',
                                   OneHotEncoder(drop='first',
                                                 handle_unknown='ignore',
                                                 sparse_output=False),
                                   [3, 1])])),
 ('columntransformer-3',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('scale', MinMaxScaler(), [0, -1])])),
 ('selectkbest',
  SelectKBest(k=8, score_func=<function chi2 at 0x7945aec9e0e0>)),
 ('decisiontreeclassifier', DecisionTreeClassifier())]

In [250]:
#fit
pipe.fit(X_train,y_train)


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [3, 1])])),
                ('columntransformer-3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  [0, -1])])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7945aec9e0e0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [251]:
# Predict
y_pred = pipe.predict(X_test)

In [252]:
y_pred

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1])

###Cross-Validation


In [254]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5, scoring='accuracy').mean()

0.7444105190584064

###Grid Search

In [268]:
print(pipe.named_steps)


{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                  transformers=[('impute age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])]), 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 [3, 1])]), 'columntransformer-3': ColumnTransformer(remainder='passthrough',
                  transformers=[('scale', MinMaxScaler(), [0, -1])]), 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x7945aec9e0e0>), 'decisiontreeclassifier': DecisionTreeClassifier()}


In [269]:
params = {
    'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5, None]
}


In [270]:
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute '
                                                                         'age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(drop='first...
                                                                                       sparse_output=False),
                                                                         [3,
                                                                          1])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         [0,
                                                                          -1])])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7945aec9e0e0>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [271]:
grid.best_score_

0.8202107751403526

In [272]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 3}

###Exporting the Pipeline

In [273]:
import pickle

In [274]:
pickle.dump(pipe,open('pipe.pkl','wb'))